In [1]:
#!/usr/bin/env python3

In [2]:
# import librarys
import pandas as pd
import subprocess
import os
from urllib.parse import urlsplit

In [3]:
# pathes
csv = '../data/database/db_updated.csv'
git_store = '../data/repositories/git/'
others_store = '../data/repositories/others/'

In [4]:
# read csv data
df = pd.read_csv(csv)

# print shape
print(df.shape)

(537, 6)


In [37]:
# helper function to create folder create_folder
def create_folder(path):
    if not os.path.exists(os.path.dirname(path)):
        try:
            os.makedirs(os.path.dirname(path))
            print(path + ' created')
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

# download function for git-repositories
def git_clone(url, index):
    # return if link is empty
    if isinstance(url, str):
        # base path
        #path = git_store+str(index).zfill(4)+'_'

        # split url
        split_url = urlsplit(url)
        
        # url formatting
        #split_url = split_url.netloc.replace('www.', '')
        
        print(url)
        
        # check if url is a git repository
        if split_url.netloc == 'github.com':
            
            author = split_url.path.split('/', 2)[1]
            git = split_url.path.split('/', 3)[2]
            git = git.replace('%20', ' ')
            
            print(author, git)

            # base path
            #path = git_store+author+git
            
            # url formatting
            url = url.replace('www.', '')
            url = url.replace('%20', ' ')
            url = url.split("blob",1)[0]
            url = url.split("tree",1)[0]

            # project path
            name = os.path.basename(os.path.normpath(url))
            path_project = os.path.join(git_store, author, git)
            
            print(path_project)

            # check if folder exists
            project_exists = os.path.isdir(path_project)

            # download if folder does not exist
            if not project_exists:
                print('---')
                print(path_project)

                # meta path
                path_meta = path_project + '/.meta/log_clone.txt'

                # git clone
                output = subprocess.getoutput("git clone {} {}".format(url, path_project))
                print(output)

                # create meta file for output
                create_folder(path_meta)
                with open(path_meta, 'w') as file:
                    file.write(output)
                    
    else:
        print('empty url on row '+str(index))

In [ ]:
# download all git repositories

for index, url in enumerate(df['Link'], start=1):
    #print(url)
    git_clone(url, index)

In [39]:
# list folders with missing '.git' to provide fixing
for folder in os.listdir(git_store):
    for subfolder in os.listdir(os.path.join(git_store, folder)):
    #subfolder = os.listdir(os.path.join(git_store, folder))[0]
        if not os.path.isdir(os.path.join(git_store, folder, subfolder, '.git')):
            path = os.path.join(git_store, folder, subfolder)
            print (path)